# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


In [3]:
y_train.shape

(50000, 1)

In [0]:
y_train = y_train.reshape(y_train.shape[0],)
y_test = y_test.reshape(y_test.shape[0],)

In [0]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

In [0]:
x_train_gt5 = x_train[y_train > 4]
y_train_gt5 = y_train[y_train > 4] - 5
x_test_gt5 = x_test[y_test > 4]
y_test_gt5 = y_test[y_test > 4] - 5

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
from keras.utils import to_categorical
y_train_lt5 = to_categorical(y_train_lt5,num_classes=5)
y_test_lt5 = to_categorical(y_test_lt5,num_classes=5)
y_train_gt5 = to_categorical(y_train_gt5,num_classes=5)
y_test_gt5 = to_categorical(y_test_gt5,num_classes=5)

In [0]:
x_train_lt5 = x_train_lt5.astype('float32')
x_test_lt5 = x_test_lt5.astype('float32')
x_train_gt5 = x_train_gt5.astype('float32')
x_train_gt5 = x_train_gt5.astype('float32')

x_train_lt5 = x_train_lt5/255
x_test_lt5 = x_test_lt5/255
x_train_gt5 = x_train_gt5/255
x_test_gt5 = x_test_gt5/255

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [54]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPool2D, BatchNormalization

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.3))
	
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(5, activation='softmax'))  # as we have splitted data into 2, so we have only 5 target classes i.e. 0 to 4

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train_lt5, y_train_lt5, validation_data=(x_test_lt5, y_test_lt5), epochs=30, batch_size=64)

Train on 25000 samples, validate on 5000 samples
Epoch 1/30
25000/25000 [==============================] - 10s 400us/step - loss: 1.1428 - acc: 0.5590 - val_loss: 0.8252 - val_acc: 0.6764
Epoch 2/30
25000/25000 [==============================] - 5s 198us/step - loss: 0.8133 - acc: 0.6807 - val_loss: 0.8048 - val_acc: 0.6932
Epoch 3/30
25000/25000 [==============================] - 5s 200us/step - loss: 0.7010 - acc: 0.7316 - val_loss: 0.8413 - val_acc: 0.6758
Epoch 4/30
25000/25000 [==============================] - 5s 200us/step - loss: 0.6302 - acc: 0.7632 - val_loss: 0.6127 - val_acc: 0.7652
Epoch 5/30
25000/25000 [==============================] - 5s 198us/step - loss: 0.5749 - acc: 0.7862 - val_loss: 0.8155 - val_acc: 0.7036
Epoch 6/30
25000/25000 [==============================] - 5s 200us/step - loss: 0.5334 - acc: 0.7985 - val_loss: 0.5520 - val_acc: 0.7944
Epoch 7/30
25000/25000 [==============================] - 5s 196us/step - loss: 0.4969 - acc: 0.8164 - val_loss: 0.8171 - 

In [55]:
model.evaluate(x_train_lt5, y_train_lt5)

25000/25000 [==============================] - 3s 129us/step


[0.09880135832309724, 0.9684]

In [56]:
model.evaluate(x_test_lt5, y_test_lt5)

5000/5000 [==============================] - 1s 130us/step


[0.43363800501823424, 0.8562]

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [59]:
for layer in model.layers:
  print(layer.name)
  if 'conv2d' in layer.name:
    layer.trainable = False

conv2d_93
do conv layer non trainable
batch_normalization_117
conv2d_94
do conv layer non trainable
batch_normalization_118
max_pooling2d_46
dropout_70
conv2d_95
do conv layer non trainable
batch_normalization_119
conv2d_96
do conv layer non trainable
batch_normalization_120
max_pooling2d_47
dropout_71
flatten_25
dense_49
batch_normalization_121
dropout_72
dense_50


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [60]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train_gt5, y_train_gt5, validation_data=(x_test_gt5, y_test_gt5), epochs=30, batch_size=64)

Train on 25000 samples, validate on 5000 samples
Epoch 1/30
25000/25000 [==============================] - 10s 392us/step - loss: 1.9635 - acc: 0.5422 - val_loss: 0.5840 - val_acc: 0.7882
Epoch 2/30
25000/25000 [==============================] - 5s 182us/step - loss: 0.6437 - acc: 0.7714 - val_loss: 0.4653 - val_acc: 0.8336
Epoch 3/30
25000/25000 [==============================] - 5s 182us/step - loss: 0.5473 - acc: 0.8022 - val_loss: 0.4127 - val_acc: 0.8522
Epoch 4/30
25000/25000 [==============================] - 4s 180us/step - loss: 0.4955 - acc: 0.8203 - val_loss: 0.3909 - val_acc: 0.8584
Epoch 5/30
25000/25000 [==============================] - 5s 182us/step - loss: 0.4667 - acc: 0.8304 - val_loss: 0.3667 - val_acc: 0.8682
Epoch 6/30
25000/25000 [==============================] - 5s 181us/step - loss: 0.4368 - acc: 0.8440 - val_loss: 0.3579 - val_acc: 0.8692
Epoch 7/30
25000/25000 [==============================] - 5s 184us/step - loss: 0.4196 - acc: 0.8500 - val_loss: 0.3433 - 

In [61]:
model.evaluate(x_train_gt5, y_train_gt5)

25000/25000 [==============================] - 3s 131us/step


[0.12200682881116867, 0.96396]

In [62]:
model.evaluate(x_test_gt5, y_test_gt5)

5000/5000 [==============================] - 1s 145us/step


[0.2930971926212311, 0.8956]

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [3]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [4]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [5]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2], dtype=int64)

In [6]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [7]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()
train_dtm = vect.fit_transform(twenty_train.data)
test_dtm = vect.transform(twenty_test.data)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [12]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_dtm, twenty_train.target)

D:\Softwares\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [18]:
predicted = lr.predict(test_dtm)

In [20]:
from sklearn import metrics

metrics.accuracy_score(twenty_test.target, predicted)

0.8868175765645806